In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
# AES parameters
NROW = 4
NCOL = 4
NGRID = NROW * NCOL
NBRANCH = NROW + 1     # AES MC branch number

In [3]:
# variable declaration
total_r = 7 # total round
start = 4   # start round, start in {0,1,2,...,total_r-1}
middle = 1  # meet in the middle round, middle in {0,1,2,...,total_r-1}, start != middle
fwd = []    # forward rounds
bwd = []    # backward rounds

if start < middle:
    fwd = list(range(start, middle))
    bwd = list(range(middle + 1, total_r)) + list(range(0, start))
else:
    bwd = list(range(middle + 1, start))
    fwd = list(range(start, total_r)) + list(range(0, middle))

In [4]:
# Declare and initialize model
m = gp.Model('model_%dx%d_%dR_Start_r%d_Meet_r%d' % (NROW, NCOL, total_r, start, middle))
m

Restricted license - for non-production use only - expires 2023-10-25


<gurobi.Model Continuous instance model_4x4_7R_Start_r4_Meet_r1: 0 constrs, 0 vars, No parameter changes>

In [5]:
# create Encode class to encode coloring scheme
class Encode(object):
    def __init__(self, x: gp.Var, y:gp.Var, z:gp.Var, w:gp.Var):
        self.b = x
        self.r = y
        self.g = z
        self.w = z

# define variables to represent state pattern
full_state = np.ndarray(shape=(total_r, NROW, NCOL), dtype=Encode)
for r in range(total_r):
    for i in range(NROW):
        for j in range(NCOL):
            full_state[r, i, j] = Encode(
                m.addVar(vtype=GRB.BINARY, name= "R%d[%d,%d]_b" %(r,i,j)),
                m.addVar(vtype=GRB.BINARY, name= "R%d[%d,%d]_r" %(r,i,j)),
                m.addVar(vtype=GRB.BINARY, name= "R%d[%d,%d]_g" %(r,i,j)),
                m.addVar(vtype=GRB.BINARY, name= "R%d[%d,%d]_w" %(r,i,j))
            )

In [6]:
S = np.ndarray(shape=(total_r, NROW, NCOL), dtype= Encode)  # store the SB state at each round
M = np.ndarray(shape=(total_r, NROW, NCOL), dtype= Encode)  # store the MC state at each round

S = full_state
for r in range(total_r):
    for i in range(NROW):
        for j in range(NCOL):
            M[r, i, j] = S[r, i, (j+i)%NCOL]    # match SB and MC through SR  

In [7]:
# add grey and white constraints
for r in range(total_r):
    for i in range(NROW):
        for j in range(NCOL):
            m.addConstr(S[r,i,j].g == gp.and_(S[r,i,j].b, S[r,i,j].r))  # grey cell
            m.addConstr(S[r,i,j].w - S[r,i,j].g + S[r,i,j].b + S[r,i,j].r == 1) # white cell 

In [8]:
# the degree of freedom at the starting position
start_df_b = np.ndarray(shape = (NROW, NCOL), dtype = gp.Var)
start_df_r = np.ndarray(shape = (NROW, NCOL), dtype = gp.Var)

for i in range(NROW):
    for j in range(NCOL):
        start_df_b = m.addVar(vtype=GRB.BINARY, name= "StartDF[%d,%d]_b" %(i,j))
        start_df_r = m.addVar(vtype=GRB.BINARY, name= "StartDF[%d,%d]_r" %(i,j))

# consumed degree of freedom along corresponding direction
consume_fwd = np.ndarray(shape = (total_r, NCOL), dtype = gp.Var)
consume_bwd = np.ndarray(shape = (total_r, NCOL), dtype = gp.Var)

for r in range(total_r):
    for j in range(NCOL):
        consume_fwd = m.addVar(vtype=GRB.BINARY, name= "R%dC%d_fwd" %(r,j))
        consume_bwd = m.addVar(vtype=GRB.BINARY, name= "R%dC%d_bwd" %(r,j))

# Intermediate values for computations on DoM
MT_n = np.empty(shape=(NCOL), dtype= gp.Var)
MT_m = np.empty(shape=(NCOL), dtype= gp.Var)
for j in range(NCOL):
    MT_n[j] = m.addVar(lb=-NROW, ub=NROW, name="MT_n_c" + ('[%d]' % j))
    MT_m[j] = m.addVar(lb=0, ub=NROW, vtype=GRB.INTEGER, name="MT_m_c" + ('[%d]' % j))

In [9]:
final_df_b = m.addVar(lb=1, vtype=GRB.INTEGER, name="FinalDF_b")
final_df_r = m.addVar(lb=1, vtype=GRB.INTEGER, name="FinalDF_r")
match_df = m.addVar(lb=1, vtype=GRB.INTEGER, name="MatchDF")
obj = m.addVar(lb=1, vtype=GRB.INTEGER, name="Obj")